## Zero-shot without classes incoporated

In [ ]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryPrecisionRecallCurve

def clip_pred(imgs):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    inputs = processor(
        text=["a synthetic image created by AI", "a real image taken by a human"],
        images=imgs,
        return_tensors="pt",
        padding=True
    )

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    prob = logits_per_image.softmax(dim=1)  
    return prob

In [ ]:
def load_images_from_folder(folder):
    images = []
    
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
        
    
    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels

In [ ]:
def metrics(ys, ts):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    acc.update(ys, ts)
    f1.update(ys, ts)
    cm.update(ys, ts)
    return acc.compute(), f1.compute(), cm.compute()


In [ ]:
folder_path = r'C:\Users\vikto\Documents\GitHub\DTU_repo\deep_learning\4_Convolutional\images'
images, labels = load_images_from_folder(folder_path)
probs = clip_pred(images)
metrics(probs[:,1], labels)

### HPC adjusted zero-shot no classes

In [ ]:
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix

# Enable debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["PYTHONFAULTHANDLER"] = "1"

# Log GPU details
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
    print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available. Check your GPU setup.")

def clip_pred(imgs, model, processor):
    """
    Perform prediction using the CLIP model.
    """
    inputs = processor(
        text=["a synthetic image created by AI", "a real image taken by a human"],
        images=imgs,
        return_tensors="pt",
        padding=True
    )
    
    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # Image-text similarity score
    prob = logits_per_image.softmax(dim=1)  # Probability over classes
    return prob

def load_images_from_folders(fake_folder, real_folder):
    """
    Load images and their corresponding labels from specified folders.
    """
    images = []
    labels = []
    
    # Load FAKE images
    for filename in os.listdir(fake_folder):
        img_path = os.path.join(fake_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(0)  # Label for "FAKE"
    
    # Load REAL images
    for filename in os.listdir(real_folder):
        img_path = os.path.join(real_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(1)  # Label for "REAL"
    
    return images, torch.tensor(labels, dtype=torch.int)

def evaluate_model(images, labels, batch_size=64):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    for i in range(0, len(images), batch_size):
        batch_imgs = images[i:i+batch_size]
        batch_probs = clip_pred(batch_imgs, model, processor)
        probs.append(batch_probs)
    
    probs = torch.cat(probs, dim=0)  # Combine all batches
    preds = torch.argmax(probs, dim=1)  # Predicted labels
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    
    acc.update(preds, labels)
    f1.update(preds, labels)
    cm.update(preds, labels)
    
    print(f"Accuracy: {acc.compute().item():.4f}")
    print(f"F1 Score: {f1.compute().item():.4f}")
    print(f"Confusion Matrix: {cm.compute()}")
    return acc.compute(), f1.compute(), cm.compute()

# Paths to the datasets
fake_folder = r'/dtu/blackhole/18/160664/test/FAKE/'
real_folder = r'/dtu/blackhole/18/160664/test/REAL/'

# Load the images and labels
images, labels = load_images_from_folders(fake_folder, real_folder)
print(f"Loaded {len(images)} images.")
print(f"Labels: {labels}")

# Evaluate the model
evaluate_model(images, labels)

## Zero-shot with classes implemented

In [ ]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC

def clip_pred(imgs_class, class_type, model, processor):

    # Process all images and prompts in a single batch
    inputs = processor(
        text= ['A human-made photo of a' + str(class_type), 'A synthetic computer-generated photo of a' + str(class_type)],  # Prompts for each image
        images=imgs_class,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to("cpu") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # Image-text similarity score
    prob = logits_per_image.softmax(dim=1)  # Probability over classes
    return prob[:,1]


def load_images_from_folder(folder):
    images_class = {'airplane': [[],[]], 'automobile': [[],[]], 'bird': [[],[]], 'cat': [[],[]], 'deer': [[],[]], 'dog': [[],[]], 'frog': [[],[]], 'horse': [[],[]], 'ship': [[],[]], 'truck': [[],[]]}
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}

    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")

            for c in class_types.keys():
                if c in filename:
                    images_class[class_types.get(c)][0].append(img)
                    images_class[class_types.get(c)][1].append(0)
                    break
            if all(c not in filename for c in class_types.keys()):
                images_class['airplane'][0].append(img)
                images_class['airplane'][1].append(0)

    return images_class

def evaluate_model(imgs_class):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cpu")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    labels = []
    for i in imgs_class:
        batch_probs = clip_pred(imgs_class[i][0], i,  model, processor)
        probs.extend(batch_probs)
        labels.extend(imgs_class[i][1])

    probs = torch.tensor(probs)
    labels = torch.tensor(labels, dtype=torch.int)
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    
    acc.update(probs, labels)
    f1.update(probs, labels)
    cm.update(probs, labels)
    auroc.update(probs, labels)
    
    accuracy = acc.compute()
    f1_score = f1.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()
    
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")


folder_path = r'C:\Users\vikto\Documents\GitHub\Feature-analysis-using-CLIP-model\images'
imgs_class = load_images_from_folder(folder_path)
# Evaluate the model
evaluate_model(imgs_class)



### 

### Non HPC adjusted zero-shot with classes

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC

def clip_pred(imgs, imgs_class):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Create prompts for all images
    prompts = []
    for img_class in imgs_class:
        prompts.append("a synthetic image of a " + str(img_class))
        prompts.append("a real image of a " + str(img_class))

    # Process all images and prompts in a single batch
    inputs = processor(
        text=prompts,
        images=imgs * 2,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    # Pass the batch to the model
    outputs = model(**inputs)
    logits = outputs.logits_per_image  # this is the image-text similarity score

    # Reshape logits to separate synthetic and real image logits
    logits = logits.view(len(imgs), 2, -1).mean(dim=2)

    prob = logits.softmax(dim=1)
    return prob

def load_images_from_folder(folder):
    images = []
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels, imgs_class

def metrics(ys, ts):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    acc.update(ys, ts)
    f1.update(ys, ts)
    cm.update(ys, ts)
    auroc.update(ys, ts)

    return acc.compute(), f1.compute(), cm.compute(), auroc.compute()

folder_path = r'C:\Users\vikto\Documents\GitHub\Feature-analysis-using-CLIP-model\images'
images, labels, imgs_class = load_images_from_folder(folder_path)
probs = clip_pred(images, imgs_class)
metrics(probs[:, 1], labels)

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve


def clip_pred(imgs, imgs_class):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Create prompts for all images
    prompts = []
    for img_class in imgs_class:
        prompts.append("a synthetic image of a " + str(img_class))
        prompts.append("a real image of a " + str(img_class))

    # Process all images and prompts in a single batch
    inputs = processor(
        text=prompts,
        images=imgs * 2,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    # Pass the batch to the model
    outputs = model(**inputs)
    logits = outputs.logits_per_image  # this is the image-text similarity score

    # Reshape logits to separate synthetic and real image logits
    logits = logits.view(len(imgs), 2, -1).mean(dim=2)

    prob = logits.softmax(dim=1)
    return prob

def load_images_from_folder(folder):
    images = []
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels, imgs_class

def evaluate_metrics(probs, labels):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()

    acc.update(probs[:, 1], labels)
    f1.update(probs[:, 1], labels)
    cm.update(probs[:, 1], labels)
    auroc.update(probs[:, 1], labels)

    accuracy = acc.compute()
    f1_score = f1.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()

    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")

    # Plot ROC Curve
    fpr, tpr, _ = roc_curve(labels.detach().numpy(), probs[:, 1].detach().numpy())
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, marker='.', label='ROC Curve (AUC = {:.2f})'.format(auroc_score))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.grid(True)
    plt.show()

folder_path = r'C:\Users\vikto\Documents\GitHub\Feature-analysis-using-CLIP-model\images'
images, labels, imgs_class = load_images_from_folder(folder_path)
probs = clip_pred(images, imgs_class)
evaluate_metrics(probs, labels)

### Adjusted class code for HPC

#### For running fake and real seperately:

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC

# Enable debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["PYTHONFAULTHANDLER"] = "1"

# Log GPU details
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
    print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available. Check your GPU setup.")

def clip_pred(imgs_class, class_type, model, processor):

    # Process all images and prompts in a single batch
    inputs = processor(
        text= ['A human-made photo of a' + str(class_type), 'A synthetic computer-generated photo of a' + str(class_type)],  # Prompts for each image
        images=imgs_class,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to("cpu") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
        image_features = outputs.get_image_features(outputs['pixel_values'], hidden_states=False)
    logits_per_image = outputs.logits_per_image  # Image-text similarity score
    prob = logits_per_image.softmax(dim=1)  # Probability over classes
    return prob[:,1], image_features

def load_images_from_folder(folder):
    images_class = {'airplane': [[],[]], 'automobile': [[],[]], 'bird': [[],[]], 'cat': [[],[]], 'deer': [[],[]], 'dog': [[],[]], 'frog': [[],[]], 'horse': [[],[]], 'ship': [[],[]], 'truck': [[],[]]}
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}

    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")

            for c in class_types.keys():
                if c in filename:
                    images_class[class_types.get(c)][0].append(img)
                    images_class[class_types.get(c)][1].append(0)
                    break
            if all(c not in filename for c in class_types.keys()):
                images_class['airplane'][0].append(img)
                images_class['airplane'][1].append(0)

    return images_class

def evaluate_model(imgs_class):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cpu")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    labels = []
    for i in imgs_class:
        batch_probs = clip_pred(imgs_class[i][0], i,  model, processor)
        probs.extend(batch_probs)
        labels.extend(imgs_class[i][1])

    probs = torch.tensor(probs)
    labels = torch.tensor(labels, dtype=torch.int)
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    
    acc.update(probs, labels)
    f1.update(probs, labels)
    cm.update(probs, labels)
    auroc.update(probs, labels)
    
    accuracy = acc.compute()
    f1_score = f1.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()
    
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")



# Paths to the datasets
folder_path = r'C:\Users\vikto\Documents\GitHub\Feature-analysis-using-CLIP-model\images'

# Load the images and labels
imgs_class= load_images_from_folder(folder_path)

# Evaluate the model
evaluate_model(imgs_class)

#### Fake and real at the same time:

In [ ]:
import torch
from PIL import Image
import os
from transformers import CLIPProcessor, CLIPModel
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC

# Enable debugging
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["PYTHONFAULTHANDLER"] = "1"

# Log GPU details
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory Allocated: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB")
    print(f"CUDA Memory Reserved: {torch.cuda.memory_reserved() / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available. Check your GPU setup.")

def clip_pred(imgs, imgs_class, model, processor):
    """
    Perform prediction using the CLIP model.
    """
    # Create prompts for all images
    prompts = []
    for img_class in imgs_class:
        prompts.append("a synthetic image of a " + str(img_class))
        prompts.append("a real image of a " + str(img_class))

    # Process all images and prompts in a single batch
    inputs = processor(
        text=prompts,
        images=imgs * 2,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
    logits = outputs.logits_per_image  # this is the image-text similarity score

    # Reshape logits to separate synthetic and real image logits
    logits = logits.view(len(imgs), 2, -1).mean(dim=2)

    prob = logits.softmax(dim=1)
    return prob

def load_images_from_folders(fake_folder, real_folder):
    images = []
    labels = []
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    # Load FAKE images
    for filename in os.listdir(fake_folder):
        img_path = os.path.join(fake_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(0)  # Label for "FAKE"

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    # Load REAL images
    for filename in os.listdir(real_folder):
        img_path = os.path.join(real_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            images.append(img)
            labels.append(1)  # Label for "REAL"

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
            else:
                imgs_class.append('airplane')

    return images, torch.tensor(labels, dtype=torch.int), imgs_class

def evaluate_model(images, labels, imgs_class, batch_size=64):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    for i in range(0, len(images), batch_size):
        batch_imgs = images[i:i+batch_size]
        batch_classes = imgs_class[i:i+batch_size]
        batch_probs = clip_pred(batch_imgs, batch_classes, model, processor)
        probs.append(batch_probs)
    
    probs = torch.cat(probs, dim=0)  # Combine all batches
    preds = torch.argmax(probs, dim=1)  # Predicted labels
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    
    acc.update(preds, labels)
    f1.update(preds, labels)
    cm.update(preds, labels)
    auroc.update(preds, labels)
    
    accuracy = acc.compute()
    f1_score = f1.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()
    
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")

# Paths to the datasets
fake_folder = r'/dtu/blackhole/18/160664/test/FAKE/'
real_folder = r'/dtu/blackhole/18/160664/test/REAL/'

# Load the images and labels
images, labels, imgs_class = load_images_from_folders(fake_folder, real_folder)
print(f"Loaded {len(images)} images.")
print(f"Labels: {labels}")

# Evaluate the model
evaluate_model(images, labels, imgs_class)

In [89]:
l = [[1,2,3], [4,5,6], [7,8,9]]
j=[]
for i in l:
    j.extend(i)
print(j)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


## Extracting image features and applying FFNN

In [62]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC
from tqdm import tqdm

def clip_pred(imgs_class, class_type, model, processor):

    # Process all images and prompts in a single batch
    inputs = processor(
        text= ['A human-made photo of a' + str(class_type), 'A synthetic computer-generated photo of a' + str(class_type)],  # Prompts for each image
        images=imgs_class,  # Duplicate images to match the number of prompts
        return_tensors="pt",
        padding=True
    )

    inputs = {k: v.to("cpu") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
        img_features = model.get_image_features(pixel_values=inputs['pixel_values'])
    logits_per_image = outputs.logits_per_image  # Image-text similarity score
    prob = logits_per_image.softmax(dim=1)  # Probability over classes
    pred = (prob[:,1] > 0.5).int()
    return pred, img_features


def load_images_from_folder(folder):
    images_class = {'airplane': [[],[]], 'automobile': [[],[]], 'bird': [[],[]], 'cat': [[],[]], 'deer': [[],[]], 'dog': [[],[]], 'frog': [[],[]], 'horse': [[],[]], 'ship': [[],[]], 'truck': [[],[]]}
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}

    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")

            for c in class_types.keys():
                if c in filename:
                    images_class[class_types.get(c)][0].append(img)
                    images_class[class_types.get(c)][1].append(0)
                    break
            if all(c not in filename for c in class_types.keys()):
                images_class['airplane'][0].append(img)
                images_class['airplane'][1].append(0)
    

    return images_class

def evaluate_model(imgs_class):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cpu")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Process images in mini-batches
    probs = []
    labels = []
    image_features = []
    for i in imgs_class:
        batch_probs, batch_features = clip_pred(imgs_class[i][0], i,  model, processor)
        probs.extend(batch_probs)
        labels.extend(imgs_class[i][1])
        for f in batch_features:
            image_features.append(f)
    
    probs = torch.tensor(probs)
    labels = torch.tensor(labels, dtype=torch.int64)
    dataset = {'features': image_features, 'labels': labels}
    
    # Calculate metrics
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()
    
    acc.update(probs, labels)
    f1.update(probs, labels)
    cm.update(probs, labels)
    auroc.update(probs, labels)
    
    accuracy = acc.compute()
    f1_score = f1.compute()
    confusion_matrix = cm.compute()
    auroc_score = auroc.compute()
    
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print(f"Confusion Matrix: \n{confusion_matrix}")
    print(f"AUROC: {auroc_score}")

    return dataset

folder_path = r'C:\Users\vikto\Documents\GitHub\Feature-analysis-using-CLIP-model\images'
imgs_class = load_images_from_folder(folder_path)
# Evaluate the model
feature_data = evaluate_model(imgs_class)



c:\Users\vikto\miniconda3\envs\deep_learning\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Accuracy: 0.13333334028720856
F1 Score: 0.0
Confusion Matrix: 
tensor([[ 4., 26.],
        [ 0.,  0.]])
AUROC: 0.5


### Feed foward neural network on image features from encoder:

In [68]:
from torch.utils.data import DataLoader, Dataset

class dict_to_data(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels.float()

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [69]:
dataset = dict_to_data(feature_data['features'], feature_data['labels'])
train_loader = DataLoader(dataset, batch_size=10, shuffle=True, drop_last=True)

In [70]:
from torch import nn
import torch.optim as optim

num_features = len(dataset.features[0])
num_hidden = 256

class network(nn.Module):

    def __init__(self, num_features, num_hidden):

        super(network, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(num_features, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)
    

ffnn = network(num_features, num_hidden)
ffnn.to("cpu")
print(ffnn)

network(
  (net): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=1, bias=True)
    (3): Sigmoid()
  )
)


#### train loop

In [66]:
optimizer = optim.Adam(ffnn.parameters(), lr=0.0001)
loss = nn.BCELoss()

In [81]:
def train_loop(train_loader, model, loss, num_epochs):
     
    model.train()
    acc = BinaryAccuracy()

    for epoch in range(num_epochs):
        for batch_features, batch_labels in tqdm(train_loader, desc = "Training"):  
            batch_features, batch_labels = batch_features.to("cpu"), batch_labels.to("cpu")

            batch_output = model(batch_features).squeeze()
            batch_preds = (batch_output > 0.5).int()
            batch_loss = loss(batch_output, batch_labels)

            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()

            acc.update(batch_preds, batch_labels)
            batch_accuracy = acc.compute()
            acc.reset()
            
        print(f"Epoch {epoch+1}, Loss: {batch_loss}, Accuracy: {batch_accuracy}")
train_loop(train_loader, ffnn, nn.BCELoss(), 10)

Training: 100%|██████████| 3/3 [00:00<00:00, 428.38it/s]


Epoch 1, Loss: 0.6302995085716248, Accuracy: 1.0


Training: 100%|██████████| 3/3 [00:00<00:00, 544.27it/s]


Epoch 2, Loss: 0.6309584379196167, Accuracy: 1.0


Training: 100%|██████████| 3/3 [00:00<00:00, 749.61it/s]


...

Epoch 8, Loss: 0.6323121190071106, Accuracy: 1.0


Training: 100%|██████████| 3/3 [00:00<00:00, 749.79it/s]


Epoch 9, Loss: 0.6389063596725464, Accuracy: 1.0


Training: 100%|██████████| 3/3 [00:00<00:00, 750.23it/s]

Epoch 10, Loss: 0.6283644437789917, Accuracy: 1.0


#### test network

In [86]:
def test_network(test_loader, model, loss):
    model.eval()
    test_loss = 0
    test_acc = 0
    acc = BinaryAccuracy()
    test_cm = BinaryConfusionMatrix()
    test_auroc = BinaryAUROC()
    with torch.no_grad():
        for batch_features, batch_labels in tqdm(test_loader, desc = "Testing"):
            batch_features, batch_labels = batch_features.to('cpu'), batch_labels.to('cpu')

            # Forward pass
            batch_outputs = model(batch_features).squeeze()
            batch_loss = loss(batch_outputs, batch_labels)
            test_loss += batch_loss
            batch_labels = batch_labels.to(torch.int64)

            acc.update(batch_outputs, batch_labels)
            test_cm.update(batch_outputs, batch_labels)
            batch_accuracy = acc.compute()
            test_acc += batch_accuracy
            acc.reset()
            print(f'Test Loss: {batch_loss:.4f}, Test Accuracy: {batch_accuracy:.4f}')

    avg_loss = test_loss / len(test_loader)
    avg_acc =  test_acc / len(test_loader)
    cm = test_cm.compute()

    print(f'Average test loss: {avg_loss:.4f}, Average test accuracy: {avg_acc:.4f}, Confusion Matrix: {cm}')

test_network(train_loader ,model = ffnn, loss = nn.BCELoss())

Testing: 100%|██████████| 3/3 [00:00<00:00, 750.14it/s]

Test Loss: 0.6310, Test Accuracy: 1.0000
Test Loss: 0.6356, Test Accuracy: 1.0000
Test Loss: 0.6270, Test Accuracy: 1.0000
Average test loss: 0.6312, Average test accuracy: 1.0000, Confusion Matrix: tensor([[30.,  0.],
        [ 0.,  0.]])


In [ ]:
import torch
from torch import nn
import torch.optim as optim
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryAUROC
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

def clip_pred(imgs_class, class_type, model, processor):
    """
    Predict probabilities and extract features using the CLIP model.
    """
    inputs = processor(
        text=['A human-made photo of a ' + str(class_type), 
              'A synthetic computer-generated photo of a ' + str(class_type)],
        images=imgs_class,
        return_tensors="pt",
        padding=True
    )
    inputs = {k: v.to("cuda") for k, v in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
        img_features = model.get_image_features(pixel_values=inputs['pixel_values']).cpu()
    logits_per_image = outputs.logits_per_image  # Image-text similarity score
    prob = logits_per_image.softmax(dim=1)  # Probability over classes
    return prob[:, 1], img_features

def load_images_from_folder(fake_folder, real_folder):
    """
    Load images and classify them into real and fake.
    """
    images_class = {'airplane': [[], []], 'automobile': [[], []], 'bird': [[], []], 'cat': [[], []],
                    'deer': [[], []], 'dog': [[], []], 'frog': [[], []], 'horse': [[], []], 
                    'ship': [[], []], 'truck': [[], []]}
    class_types = {'(2)': 'automobile', '(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', 
                   '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}

    for filename in os.listdir(real_folder):
        img_path = os.path.join(real_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            for c in class_types.keys():
                if c in filename:
                    images_class[class_types[c]][0].append(img)
                    images_class[class_types[c]][1].append(0)  # Real
                    break
            if all(c not in filename for c in class_types.keys()):
                images_class['airplane'][0].append(img)
                images_class['airplane'][1].append(0)  # Real

    for filename in os.listdir(fake_folder):
        img_path = os.path.join(fake_folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path).convert("RGB")
            for c in class_types.keys():
                if c in filename:
                    images_class[class_types[c]][0].append(img)
                    images_class[class_types[c]][1].append(1)  # Fake
                    break
            if all(c not in filename for c in class_types.keys()):
                images_class['airplane'][0].append(img)
                images_class['airplane'][1].append(1)  # Fake
    return images_class

def evaluate_model(imgs_class):
    """
    Evaluate the CLIP model using mini-batch processing and calculate metrics.
    """
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cuda")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    probs, labels, image_features = [], [], []

    for i, class_type in enumerate(imgs_class.keys()):
        print(f"Processing class {class_type} ({i + 1}/{len(imgs_class.keys())})")
        batch_probs, batch_features = clip_pred(imgs_class[class_type][0], class_type, model, processor)
        probs.extend(batch_probs.cpu().tolist())
        labels.extend(imgs_class[class_type][1])
        image_features.extend(batch_features.cpu())
        torch.cuda.empty_cache()

    probs = torch.tensor(probs)
    labels = torch.tensor(labels, dtype=torch.int)
    dataset = {'features': image_features, 'labels': labels}

    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    auroc = BinaryAUROC()

    acc.update(probs, labels)
    f1.update(probs, labels)
    cm.update(probs, labels)
    auroc.update(probs, labels)

    print(f"Accuracy: {acc.compute():.4f}")
    print(f"F1 Score: {f1.compute():.4f}")
    print(f"Confusion Matrix: \n{cm.compute()}")
    print(f"AUROC: {auroc.compute():.4f}")

    return dataset

def train_loop(train_loader, model, loss, optimizer, num_epochs):
    """
    Training loop for the feed-forward neural network.
    """
    model.train()
    acc = BinaryAccuracy()

    for epoch in range(num_epochs):
        epoch_loss = 0
        epoch_acc = 0
        for batch_features, batch_labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}"):
            batch_features, batch_labels = batch_features.to("cuda"), batch_labels.to("cuda")

            optimizer.zero_grad()
            batch_output = model(batch_features)
            batch_loss = loss(batch_output.squeeze(), batch_labels)
            batch_loss.backward()
            optimizer.step()

            acc.update(batch_output.squeeze(), batch_labels)
            epoch_acc += acc.compute().item()
            epoch_loss += batch_loss.item()

        avg_loss = epoch_loss / len(train_loader)
        avg_acc = epoch_acc / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}")
        acc.reset()
        torch.cuda.empty_cache()

def test_network(test_loader, model, loss):
    """
    Testing loop for the feed-forward neural network.
    """
    model.eval()
    acc = BinaryAccuracy()
    auroc = BinaryAUROC()
    cm = BinaryConfusionMatrix()
    test_loss = 0

    with torch.no_grad():
        for batch_features, batch_labels in tqdm(test_loader, desc="Testing"):
            batch_features, batch_labels = batch_features.to("cuda"), batch_labels.to("cuda")
            
            batch_outputs = model(batch_features).squeeze()
            batch_loss = loss(batch_outputs, batch_labels)
            test_loss += batch_loss.item()

            batch_outputs = torch.tensor(batch_outputs)
             batch_labels = torch.tensor(batch_labels, dtype=torch.int)

            acc.update(batch_outputs, batch_labels)
            cm.update(batch_outputs, batch_labels)
            auroc.update(batch_outputs, batch_labels)

    avg_loss = test_loss / len(test_loader)
    print(f"Average Test Loss: {avg_loss:.4f}")
    print(f"Accuracy: {acc.compute():.4f}")
    print(f"AUROC: {auroc.compute():.4f}")
    print(f"Confusion Matrix:\n{cm.compute()}")

class dict_to_data(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels.float()

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

class network(nn.Module):
    def __init__(self, num_features, num_hidden):
        super(network, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(num_features, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# File paths
fake_train = r"C:\Users\Bruger\Documents\School\Feature-analysis-using-CLIP-model\images\fake_train"
real_train = r"C:\Users\Bruger\Documents\School\Feature-analysis-using-CLIP-model\images\real_train"

# Load training data
imgs_class = load_images_from_folder(fake_train, real_train)
feature_data_train = evaluate_model(imgs_class)
train_dataset = dict_to_data(feature_data_train['features'], feature_data_train['labels'])
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)

# Train the network
num_features = len(train_dataset.features[0])
ffnn = network(num_features, 256).to("cuda")
train_loop(train_loader, ffnn, nn.BCELoss(), optim.Adam(ffnn.parameters(), lr=1e-4), num_epochs=4)

# Test the network
fake_test = r"C:\Users\Bruger\Documents\School\Feature-analysis-using-CLIP-model\images\fake_test"
real_test = r"C:\Users\Bruger\Documents\School\Feature-analysis-using-CLIP-model\images\real_test"
test_imgs_class = load_images_from_folder(fake_test, real_test)
feature_data_test = evaluate_model(test_imgs_class)
test_dataset = dict_to_data(feature_data_test['features'], feature_data_test['labels'])
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, drop_last=True)

test_network(test_loader, ffnn, nn.BCELoss())

Processing class airplane (1/10)
Processing class automobile (2/10)
Processing class bird (3/10)
Processing class cat (4/10)
Processing class deer (5/10)
Processing class dog (6/10)
Processing class frog (7/10)
Processing class horse (8/10)
Processing class ship (9/10)
Processing class truck (10/10)
Accuracy: 0.5000
F1 Score: 0.6667
Confusion Matrix: 
tensor([[ 0., 30.],
        [ 0., 30.]])
AUROC: 0.5000


Training Epoch 1/4: 100%|██████████| 30/30 [00:00<00:00, 540.17it/s]


Epoch 1, Loss: 0.7100, Accuracy: 0.4490


Training Epoch 2/4: 100%|██████████| 30/30 [00:00<00:00, 548.83it/s]


Epoch 2, Loss: 0.6987, Accuracy: 0.5014


...

Training Epoch 4/4: 100%|██████████| 30/30 [00:00<00:00, 572.04it/s]

Epoch 4, Loss: 0.6990, Accuracy: 0.4836


Processing class airplane (1/10)
Processing class automobile (2/10)
Processing class bird (3/10)
Processing class cat (4/10)
Processing class deer (5/10)
Processing class dog (6/10)
Processing class frog (7/10)
Processing class horse (8/10)
Processing class ship (9/10)
Processing class truck (10/10)
Accuracy: 0.5000
F1 Score: 0.6667
Confusion Matrix: 
tensor([[ 0., 30.],
        [ 0., 30.]])
AUROC: 0.5000


Testing:   0%|          | 0/30 [00:00<?, ?it/s]C:\Users\Bruger\AppData\Local\Temp\ipykernel_11104\409205293.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_outputs = torch.tensor(batch_outputs)
C:\Users\Bruger\AppData\Local\Temp\ipykernel_11104\409205293.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_labels = torch.tensor(batch_labels, dtype=torch.int)
Testing: 100%|██████████| 30/30 [00:00<00:00, 1034.80it/s]

Average Test Loss: 0.6934
Accuracy: 0.5000
AUROC: 0.5000
Confusion Matrix:
tensor([[ 5., 25.],
        [ 5., 25.]])
